Conversion between photon cm$^{-2}$ s$^{-1}$ sr$^{-1}$ and erg cm$^{-2}$ s$^{-1}$ arcsec$^{-2}$.

In [4]:
from astropy import constants as const
from astropy import units as u
import eagle_constants_and_units as cons
import numpy as np

photon_wavelength = 656.3e-9*u.m

SB_cgs = 10**-22 *u.erg / u.arcsec**2 / u.cm**2 / u.s
SB_ph = SB_cgs * photon_wavelength/(const.h.to('erg s') * const.c.to('m/s')) * (206265.*u.arcsec)**2/u.sr

print('The surface brightness, %s, in cgs units is: %s'%(SB_ph.round(1),SB_cgs))

radians = 1.
degrees = np.pi/180.
arcmin = 1./60. * degrees 
arcsec = 1./3600. * degrees

g = 1.
kg = 10.**3 * g

s = 1.
Hz = 1/s

cm = 1.
nm = 10.**-7 * cm
mum = 10**-4 * cm


#S_erg = S_Jy *(10**-23 * u.erg / (u.s * u.Hz * u.cm**2) / u.Jy )
erg = 1.
Jy = 10**-23 * erg/(s*cm**2) *1/Hz  

def getSB_ph(SB_cgs,photon_wavelength=656.3e-9*u.m):
    return SB_cgs * photon_wavelength/(const.h.to('erg s') * const.c.to('m/s')) * (206265.*u.arcsec)**2/u.sr
def getSB_cgs(SB_ph,photon_wavelength=656.3e-9*u.m):
    return SB_ph / (photon_wavelength/(const.h.to('erg s') * const.c.to('m/s')) * (206265.*u.arcsec)**2/u.sr)

print getSB_ph(1.62*10**-19 *u.erg / u.arcsec**2 / u.cm**2 / u.s, photon_wavelength=6583*10**-10*u.m)
print getSB_cgs(1417.608)

print 'this: %s'%(2.7e2 * (6563./10**3) * (1./1.25))

def SB_photonscgs_to_ABmagarcsec(sb):
    fld = sb * cons.planck * (4*np.pi)/3600**2 # flux density: energy / (m**2 s)  /frequency (/arcmin**2)  sufrace brightness: photons / (cm**2 s) /sr       
    return -2.5*np.log10(fld/(3631.*Jy))

print SB_photonscgs_to_ABmagarcsec(SB_ph.value)

print SB_photonscgs_to_ABmagarcsec(0.01)
print SB_photonscgs_to_ABmagarcsec(10)

# Ly alpha feature at redshift z ~ 6.6
print getSB_ph(1.*10**-18 *u.erg / u.arcsec**2 / u.cm**2 / u.s, photon_wavelength=1216*10**-10*u.m*(1.+2.))

# Martin et al 2014: 1000 ph/cm^2/s/sr ~ 10^-19 erg/cm^2/s/arcsec^2
print getSB_ph(1.*10**-19 *u.erg / u.arcsec**2 / u.cm**2 / u.s, photon_wavelength=5000*10**-10*u.m)
print getSB_cgs(1400,photon_wavelength=5000*10**-10*u.m)

# Fumagalli et al 2017: 1.2*10**-19 *u.erg / u.arcsec**2 / u.cm**2
print getSB_ph(1.2*10**-19 *u.erg / u.arcsec**2 / u.cm**2 / u.s,photon_wavelength=656.3e-9*u.m)

The surface brightness, 1.4 1 / (cm2 s sr), in cgs units is: 1e-22 erg / (arcsec2 cm2 s)
2284.09023733 1 / (cm2 s sr)
1.00850803631e-19 erg sr / arcsec2
this: 1417.608
31.5107001353
36.8803921157
29.3803921157
7813.20496845 1 / (cm2 s sr)
1070.88883888 1 / (cm2 s sr)
1.30732523225e-19 erg sr / arcsec2
1686.77842789 1 / (cm2 s sr)


Conversion between photon/pixel and mag arcsec$^{-2}$ (from Abraham et al. 2014):

>The surface brightness in equation (4), S, can be cast into units of
magnitude per arcsecsecond squared (mag arcsec$^{-2}$) by noting that
$S = F \times 10^{-0.4\mu} $
, where F is the flux zero point for the
filter and $\mu$ is the mean surface brightness in magnitude units in the bandpass.

>In the AB system, the zeroth magnitude calibrator is defined in frequency space as 3631 Jy in all bands, so after conversion back to wavelength space, 
$F = (3631$ Jy/arcsec$^{2}) \times c / \lambda^2$
,where $\lambda$ is the effective wavelength of the filter.

In [4]:
SB_mag = 21.5 #* u.mag / u.arcsec**2
F_zp = 3631. * u.Jy / u.arcsec**2
eff_wavelen = 475. * 10**-9 * u.m # just by eyeballing the G filter transmission spectrum

SB_ph = F_zp * const.c / eff_wavelen**2 * 10**(-0.4*SB_mag)

print('The surface brightness, %s mag/arcsec^2, is: %s'%(SB_mag, SB_ph))

The surface brightness, 21.5 mag/arcsec^2, is: 1.21188031907e+16 Jy / (arcsec2 m s)


Just ignore the /arcsec^2 for now since we can basically ignore that.

### Sky Brightness

The sky brightness in g-band is approximately 21.5 mag/arcsec^2.  We are going through a few steps to get this into ph/s so that we can compare it to the sky background calculated for the Halpha filter.

Some useful references:
- http://classic.sdss.org/dr6/algorithms/fluxcal.html#sdss2ab
- https://www.astro.umd.edu/~ssm/ASTR620/mags.html
- http://www.stsci.edu/~strolger/docs/UNITS.txt
- http://www.astro.caltech.edu/~george/ay20/Ay20-Lec3x.pdf
- http://www.astro.caltech.edu/~george/ay122/Bessel2005ARAA43p293.pdf
- https://www.gemini.edu/sciops/instruments/midir-resources/imaging-calibrations/fluxmagnitude-conversion


In [26]:
SB_mag = 21.5 #* u.mag 
F_zp = 3631. * u.Jy
eff_wavelen = 475e-9 * u.m # just by eyeballing the G filter transmission spectrum

S_Jy = F_zp * 10**(-0.4*SB_mag)
print S_Jy

S_erg = S_Jy *(10**-23 * u.erg / (u.s * u.Hz * u.cm**2) / u.Jy )
print S_erg

9.12065963281e-06 Jy
9.12065963281e-29 erg / cm2




$$f = \int S_\nu d\nu \approx S_\nu \Delta\nu$$

$$c = \nu\lambda$$

$$\Delta\nu \approx -c \frac{\Delta\lambda}{\lambda^2}$$

S_erg_tot_hz is in the units:  erg/ Hz / s / cm^2 

On the webpage, the flux density units are: ergs / s / cm^2 / A (where I think that A is Angstroms)

That means we need to convert between /Hz and /A.

$$\lambda*f_\lambda = \nu*f_\nu$$
so
$$f_\nu = f_\lambda*(\lambda/\nu) = f_\lambda*\lambda^2/c$$.

In [47]:
bandpass_in_lambda = 150e-9 * u.m #, approximate for our g-band. from the dragonfly paper

bandpass_in_Hz = bandpass_in_lambda / eff_wavelen**2 * 3e8 *u.m/u.s
print bandpass_in_Hz

S_erg_hz = S_Jy *(10**-23 * u.erg / (u.Hz* u.s * u.cm**2) / u.Jy ) #* bandpass_in_Hz

print "All of these are also per arcsec^2 since that is what the base magnitude of the sky, 21.5 mag/arsec^2, is."
S_erg_hz

1.9944598338e+14 1 / s
All of these are also per arcsec^2 since that is what the base magnitude of the sky, 21.5 mag/arsec^2, is.


<Quantity 9.120659632811291e-29 erg / cm2>

In [48]:
S_erg_A = S_erg_hz * (3e8 *u.m/u.s / eff_wavelen**2) / 10**10
S_erg_A

<Quantity 1.2127192863571797e-17 erg / (cm2 m s)>

So this value is comparable with the ones found on the webpage http://www.stsci.edu/~strolger/docs/UNITS.txt (where they state that a surface brightness of ~21.5 corresponds to about 1.5 x 10^-6 ph/cm^2/s/Angstrom/arcsec^2):

In [46]:
"""
Typical surface brightnesses:

          mag      ph/cm^2/s/A/arcsec^2 W/m^2/Hz/arcsec^2 ergs/s/cm^2/A/arcsec^2
 -------------------------------------------------------------------------------
 CTIO  U  22.4             8.3e-07           1.99e-32           4.61e-18
 CTIO  B  22.9 
 KPNO  B  23.0             9.2e-07           2.68e-32           4.16e-18
 HST   B  24.0 NEP         3.7e-07
 CTIO  V  22.0                               5.75e-32           5.70e-18
 KPNO  V  21.9             1.7e-06
 HST   V  23.3 NEP         4.8e-07                              1.73e-18
 HST   V  22.1 antisolar   1.4e-06
 CTIO  R  21.2             2.4e-06           1.02e-31           7.46e-18
 HST   Rc 22.8 NEP         5.6e-07
 CTIO  I  19.9             5.3e-06           2.81e-31           1.35e-17
 UKIRT I  18.6
 HST   Ic 22.4 NEP         5.5e-07
 UKIRT Z  17.6
 VLT   Z  18.8 +/- 0.5
 VISTA Z  18.5             9.0e-06
 LCO   J  15.9             8.3e-05           7.04e-30           1.35e-16
 UKIRT J  15.6
 VLT   J' 16.2 (1.16 - 1.32 micron)
 LCO   H  14.0             2.3e-04           2.64e-29           2.90e-16
 UKIRT H  13.6
 LCO   K  11.8             8.6e-04           1.24e-28           7.68e-16
 UKIRT K  13.4
""";

The next step is to convert it to ph / cm^2 / s / A:

$$E = \frac{hc}{\lambda}$$

In [50]:
SB_ph_A = S_erg_A * photon_wavelength/(const.h.to('erg s') * const.c.to('m/s'))
SB_ph_A

<Quantity 4.006692326599679e-06 1 / (cm2 m s)>

SB_ph_A is also consistent with the values in the above table, though is a little bit on the high end (2 - 3 times larger than one might expect, but I think in this rough estimate it is close enough to make me happy!).

### Halpha narrow-band sky brightness

If we convert that into m from cm and times by the Halpha bandwidth of 3 nm = 30 Angstroms, we get:

In [59]:
SB_ph_halpha = SB_ph_A * (100*u.cm)**2/u.m**2 * (30*u.m)
SB_ph_halpha

<Quantity 1.2020076979799035 1 / (m2 s)>

The units of SB_ph_halpha are photons / m^2 / s / arcsec^2

For my Halpha calculation, I found a background almost identical to this: 1.473626 photon/s/arcsec^2/m^2!

### G-band sky brightness

For the g-band, instead of multiplying by 30 Angstroms, we should multiply by 1500 Angstroms, which gives us:

In [57]:
SB_ph_A * (100.*u.cm)**2/u.m**2 * (1500.*u.m)

<Quantity 60.10038489899517 1 / (m2 s)>

So let's try using this as the background in the noise-adding script and then compare it to the counts that we detect in an actual Dragonfly image!